# RAG示例

**从本地目录加载PDF文件，使用huggingface上的向量模型和Chroma数据库**

In [99]:
import os
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import DirectoryLoader, PyPDFLoader

class DataBase:
    def __init__(self, model_name: str, data_path: str, database_path: str) -> None:
        """
        model_name: 检索器使用模型
        data_path: 存放pdf的文件夹路径
        database_path: 向量数据库路径
        """
        self.__embedding_model = self.__load_model(model_name)
        if os.path.exists(database_path):
            self.embedding_database = Chroma(persist_directory=database_path, embedding_function=self.__embedding_model)
        else:
            self.embedding_database = self.__init_database(data_path, database_path)
    
    def __load_model(self, model_name):
        """
        在 HuggingFaceEmbeddings 内部通过 sentence_transformers.SentenceTransformer 类的构造函数加载。
        sentence_transformers.SentenceTransformer 类的构造函数接受一个 model_name_or_path 参数，
        这个参数可以是模型的名称，也可以是模型的本地路径。
        
        因此，要从本地路径加载，直接将模型路径指定给 model_name 即可。
        """
        embedding_model = HuggingFaceEmbeddings(
            model_name=model_name,
            model_kwargs={"device": "cpu"},
            encode_kwargs={"normalize_embeddings": True},
        )
        return embedding_model

    def __init_database(self, data_path, database_path):
        loader = DirectoryLoader(
            data_path,
            glob="*.pdf",#遍历pdf文件
            recursive=True,
            show_progress=True, # 显示进度
            use_multithreading=True, # 使用多线程
            loader_cls=PyPDFLoader, # 使用加载数据的方式
            silent_errors=True, # 遇到错误继续
        )
        docs = loader.load()
        print("-"*20, "reading docs")    
        print(docs)
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1024,
            chunk_overlap=64
        )
        
        print("-"*20, "chromadb init")    
        embedding_database = Chroma.from_documents(
            docs,
            self.__embedding_model,
            # persist_directory=database_path,
        )
        print("-"*20, "init ok")    
        embedding_database.persist()
        print("-"*20, "persist")    
        return embedding_database

    def search_documents_by_chroma(self, query: str, top_k=5):
        docs = self.embedding_database.similarity_search(query, k=top_k)
        return docs


In [103]:
model_name = '../models/GanymedeNil/text2vec-large-chinese/snapshots/064717f2acfd7253bea91079d59b82e50b58c886'
data_path = '../data'
database_path= '../data/chroma'

EmbeddingBase = DataBase(
	model_name=os.path.abspath(model_name),
	data_path=os.path.abspath(data_path),
	database_path=os.path.abspath(database_path)
)

No sentence-transformers model found with name /Users/xuehongwei/github/ai-book/models/GanymedeNil/text2vec-large-chinese/snapshots/064717f2acfd7253bea91079d59b82e50b58c886. Creating a new one with MEAN pooling.
100%|██████████| 1/1 [00:00<00:00, 12.17it/s]


-------------------- reading docs
[Document(page_content='电子产品招标资质要求  \n \n一、总述 \n为确保与本公司合作的供应商具备良好的信誉、经营能力和稳定的财务状况，我们制定了一\n系列详细的招标资质要求。每一位意向供应商必须满足以下标准方可参与投标。  \n二、基本资质要求  \n1. 营业执照：供应商必须具备合法有效的营业执照。  \n2. 税务登记证：必须具备税务登记证，并符合国家税收法律法规。  \n3. 组织机构代码证：供应商应拥有有效的组织机构代码证。  \n三、经营和财务能力  \n1. 月销售额：与我方合作的月份，经我方销售的产品月总价不得低于人民币3万元。 \n2. 稳定的财务状况：供应商应具备健全和稳定的财务状况，以及良好的信用记录。  \n3. 流动资金：供应商应具备足够的流动资金来支持合同执行。  \n四、产品和服务质量  \n1. 产品质量认证： 供应商提供的产品必须符合国家及行业标准， 并通过相关质量认证。  \n2. 服务标准：供应商应具备高标准的服务体系，并承诺在合作期间提供持续、稳定、\n优质的服务。 \n五、行业经验和声誉  \n1. 行业经验：供应商应具有至少五年的相关行业经验。  \n2. 业务声誉：供应商应有良好的业务声誉和客户满意度记录。  \n六、社会责任和可持续性  ', metadata={'source': '/Users/xuehongwei/github/ai-book/data/rag_demo/demo.pdf', 'page': 0}), Document(page_content='1. 环境保护 ：供应商应遵循环境保护法律法规，并在经营活动中实施环保措施。  \n2. 社会责任：供应商应积极承担社会责任，遵守劳工权益法律法规，并推行公平、平\n等、安全和健康的劳动环境。  \n七、提交材料  \n意向供应商应提交以下材料：  \n1. 公司简介和业务范围介绍。  \n2. 法定代表人或授权代表的身份证明。  \n3. 营业执照、税务登记证、组织机构代码证等相关证件的复印件。  \n4. 近三年的财务报表和相关审计报告。  \n5. 近三年的月销售额报表。  \n6. 产品目录及样品。  \n7. 相关质量认证证书。  \n8. 其他支持

In [106]:
result = EmbeddingBase.search_documents_by_chroma(query='经营情况')
print(result)

Number of requested results 5 is greater than number of elements in index 3, updating n_results = 3


[Document(page_content='1. 环境保护 ：供应商应遵循环境保护法律法规，并在经营活动中实施环保措施。  \n2. 社会责任：供应商应积极承担社会责任，遵守劳工权益法律法规，并推行公平、平\n等、安全和健康的劳动环境。  \n七、提交材料  \n意向供应商应提交以下材料：  \n1. 公司简介和业务范围介绍。  \n2. 法定代表人或授权代表的身份证明。  \n3. 营业执照、税务登记证、组织机构代码证等相关证件的复印件。  \n4. 近三年的财务报表和相关审计报告。  \n5. 近三年的月销售额报表。  \n6. 产品目录及样品。  \n7. 相关质量认证证书。  \n8. 其他支持资质的相关文件。  \n八、申请和审核流程  \n1. 提交申请 ：意向供应商应按照规定提交完整的申请材料。  \n2. 初步审核 ：本公司将对提交的材料进行初步审核，确保资质符合要求。  \n3. 现场考察 ：如通过初步审核，本公司将进行现场考察，以了解供应商的实际运营情\n况。 \n4. 终审决定 ：本公司将综合考察结果，对供应商进行终审，决定是否接受投标。  \n \n提交截止日期：  [指定日期 ] \n如有任何疑问或需要进一步的信息，请及时与我们联系。  \n联系人： [联系人姓名 ] ', metadata={'page': 1, 'source': '/Users/xuehongwei/github/ai-book/data/rag_demo/demo.pdf'}), Document(page_content='电子产品招标资质要求  \n \n一、总述 \n为确保与本公司合作的供应商具备良好的信誉、经营能力和稳定的财务状况，我们制定了一\n系列详细的招标资质要求。每一位意向供应商必须满足以下标准方可参与投标。  \n二、基本资质要求  \n1. 营业执照：供应商必须具备合法有效的营业执照。  \n2. 税务登记证：必须具备税务登记证，并符合国家税收法律法规。  \n3. 组织机构代码证：供应商应拥有有效的组织机构代码证。  \n三、经营和财务能力  \n1. 月销售额：与我方合作的月份，经我方销售的产品月总价不得低于人民币3万元。 \n2. 稳定的财务状况：供应商应具备健全和稳定的财务状况，以及良好的信用记录。  \n3. 流动资金：供应商应具备

# 注意事项

## 1、有些安装包缺失时，提醒并不明确，例如 pypdf

In [107]:
# 确保安装了 pypdf，否则你无法加载PDF文件
!pip install pypdf

## 2、从 huggingface 上面获取模型

**Tokenizer + Model**

In [108]:
from transformers import AutoTokenizer
from transformers import AutoModel

AutoTokenizer.from_pretrained("GanymedeNil/text2vec-large-chinese")
AutoModel.from_pretrained("GanymedeNil/text2vec-large-chinese")

# 执行上面代码后，将 huaggingface 缓冲中的目录拷贝到 models 目录中，实现本地加载

## 3、关于 windows 路径的提醒

在 Python 中，你可以使用以下三种方式来表示 Windows 下的路径：

1. 使用 Unix 风格的斜杠（`/`）。Python 会自动将这种路径转换为当前操作系统的路径格式。这种方式的优点是它在所有操作系统中都可以工作，但是它可能会让一些习惯于 Windows 路径格式的人感到困惑。

   示例：
   ```python
   path = "C:/Users/username/Documents/file.txt"
   ```

2. 使用原始字符串（`r`）和 Windows 风格的反斜杠（`\`）。原始字符串会忽略所有的转义字符，所以你可以直接写入 Windows 风格的路径。

   示例：
   ```python
   path = r"C:\Users\username\Documents\file.txt"
   ```

3. 使用双反斜杠（`\\`）来表示一个反斜杠。在 Python 的字符串中，反斜杠是一个转义字符，所以你需要使用两个反斜杠来表示一个反斜杠。

   示例：
   ```python
   path = "C:\\Users\\username\\Documents\\file.txt"
   ```

注意，无论你使用哪种方式来表示路径，你都应该使用 `os.path` 模块来操作路径。这个模块提供了一组函数，可以让你以一种与操作系统无关的方式来操作路径。例如，`os.path.join` 函数可以将多个路径组合成一个路径，而 `os.path.exists` 函数可以检查一个路径是否存在。